<a href="https://colab.research.google.com/github/seunghuilee91/Deeplearning/blob/master/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")

# 앙상블


---



## 앙상블 학습이란?
####목적
- 여러 분류기를 하나의 메타 분류기로 연결하여 개별 분류기보다 더 좋은 **일반화** 성능을 달성

####방법
- 여러개 분류 알고리즘 사용 : 다수결 투표(Voting)
- 하나의 분류 알고리즘 이용 : 배깅(Bagging), 부스팅(Boosting)

####종류
- 투표(Majority Voting) : 동일한 훈련세트
- 배깅(Bagging) : 훈련샘플에서 알고리즘 마다 별도의 훈련세트 추출
  - 랜덤포레스트(Random Forest) 
- 부스팅(Boosting) : 샘플 뽑을 때 잘못 분류된 data 50%를 재학습, 또는 가중치 이용     **[셋 중 가장 정확도가 높음, But, 실제로 사용할 때 복잡하고 어렵다. 가중치를 어떻게 구현할 것인가?]**


## 과반수 투표(Majority Voting)
- 방법 : 동일한 훈련세트로 모델 구축


<img src="https://www.researchgate.net/publication/324014302/figure/fig2/AS:644424015040514@1530654066950/Majority-voting-algorithm.png">




## 배깅(Bagging)
- 훈련샘플에서 알고리즘마다 별도의 훈련세트 추출해서 모델 구축
- 부트스트랩(Bootstrap) 실시 : 데이터로 부터 복원추출(중복허용)을 이용

### 랜덤 포레스트(Random Forest)
- 방법
 - 배깅의 일종
 - 단일 분류 알고리즘(DT) 사용
 - 포레스트 구축 : 무작위로 예측변수 선택하여 모델 구축
 - 결과 결합 : 투표(분류), 평균화(예측)
 - 나무구조로 표현(x) -> **변수의 중요도 제공**
 ; 어떤 룰에 의해 어떻게 결정되는지 설명하기 어려운 단점있음

## 부스팅(Boosting)
- 샘플 뽑을 때 잘못 분류된 data 50%를 재학습
- AdaBoost : 전체 훈련샘플 사용하고, 잘못 분류된 data에 가중치 적용-> **더 잘맞출 수 있도록 모델을 재구축 할 수 있게**



# 코드실습


## 과반수투표 VotingClassifier


### 1. 기본 패키지 설치

In [0]:
# 1.기본
import numpy as np # numpy 패키지 가져오기
import matplotlib.pyplot as plt # 시각화 패키지 가져오기

# 2.데이터 가져오기
import pandas as pd # csv -> datafrom으로 전환


In [0]:
# 3.데이터 전처리
from sklearn.preprocessing import StandardScaler # 연속변수의 표준화
from sklearn.preprocessing import LabelEncoder # 범주형 변수 수치화

# 4.훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split

#5. 분류모델구축
from sklearn.tree import DecisionTreeClassifier # 결정트리
# from sklearn.naive_bayes import GaussianNB # 나이브 베이즈
from sklearn.neighbors import KNeighborsClassifier # K-최근접이웃
# from sklearn.linear_model import Perceptron # 퍼셉트론
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 모델
# from sklearn.svm import SVC # 서포트 벡터 머신(SVM)
from sklearn.neural_network import MLPClassifier# 다중인공신경망

#6. 앙상블모델 추가
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

#7. 모델검정
from sklearn.metrics import confusion_matrix, classification_report # 정오분류표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # 정확도, 민감도 등
from sklearn.metrics import roc_curve, auc # ROC 곡선 그리기

#8. 최적화
from sklearn.model_selection import cross_validate # 교차타당도
from sklearn.pipeline import make_pipeline # 파이프라인 구축
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import GridSearchCV # 하이퍼파라미터 튜닝
from sklearn.model_selection import cross_val_score # 교차타당도 # 추가




### 2. 데이터 가져오기
#### 2.1 데이터프레임으로 저장
- 원본데이터(csv)를 dataframe 형태로 가져오기(pandas)


In [0]:
bank_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Ch06.앙상블/UniversalBank.csv', encoding='UTF-8') # 파이썬 저장 데이터 가져오기
bank_df.head()

In [0]:
bank_df.shape

In [0]:
bank_df.keys()

2.2 Data와 target으로 분리
- 필요한 데이터만 추출
- data: X, target :Y로 분리

In [0]:
X = bank_df.drop(['ID','ZIPCode','PersonalLoan'], axis=1)
X.head()

In [0]:
y = bank_df['PersonalLoan']
np.bincount(y)

### 3. 데이터 전처리
#### 3.1 data(X) 레이블 인코딩
- 문자형 자료를 숫자(범주형)로 인코딩 -> 범주형 변수를 가변수로 처리 : One Hot Encording 
- 숫자형 자료를 표준화
- 단, 결정나무, 랜덤포레스트, 나이브베이즈 분류 : 원본데이터 유지
- 가변수 처리시 문자로 처리를 해야 변수명 구분이 쉬움

In [0]:
X['Education'] = X['Education'].replace([1,2,3],['Under','Grad','Prof'])
X.head() 

In [0]:
X.keys()

In [0]:
# 에듀케이션만 더미변수로 만들어주기
X = pd.get_dummies(X[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education','Mortgage', 'SecuritiesAccount', 'CDAccount', 'Online', 'CreditCard']],
       columns=['Education'],
       drop_first = True)  # 기본이 되는 첫번째 값을 제거함. False의 경우 변수가 너무 많아 지지낳아. 

In [0]:
X.head()

#### 3.2 Class(target) 레이블 인코딩
- class(y)가 숫자형으로 되어 있기 때문에 변환 없음


In [0]:
y

#### 4. 훈련/검증용 데이터 분할
- test_size : 검증데이터 30%
- random_state : random seed 번호 = 1
- stratify : y(클래스)의 비율에 따라 분할


In [0]:
# \ 이후에 SPACE 없어야 함
X_train, X_test, y_train, y_test = \
train_test_split(X,y,
                 test_size = 0.3,
                 random_state=1,
                 stratify=y)

#### 5. 모델구축
- 여러 분류모델 구축


In [0]:
logistic = LogisticRegression(solver='liblinear',
                              penalty='l2',
                              C=0.001,
                              random_state=1)

tree = DecisionTreeClassifier(max_depth=None,
                              criterion='entropy',
                              random_state=1)

knn = KNeighborsClassifier(n_neighbors=1,
                           p=2,
                           metric='minkowski')

voting_estimators = [('logistic', logistic),('tree',tree),('knn',knn)]

# 요 문장이 핵심. voting_estimators 를 상단에서 변수화해서 가져옴. 간단하게 쓰기 위해서!
voting = VotingClassifier(estimators = voting_estimators,
                          voting='soft')


### 6. 모델검정
#### 6.1 AUC검정


In [0]:
clf_labels = ['Logistic regression', 'Decision tree', 'KNN', 'Majority voting']
all_clf = [logistic,tree,knn,voting]

for clf, label in zip(all_clf, clf_labels):   # clf 모델, zip 변수 두개를 동시에 돌려주면서 반복할 때 
    scores = cross_val_score(estimator=clf,  # 교차분석표
                             X=X_train,
                             y=y_train,
                             cv=10,               #교차분석표를 10개로 나눠서 
                             scoring='roc_auc')
    print("ROC AUC: %0.3f (+/- %0.3f) [%s]"
    % (scores.mean(), scores.std(), label))

#### 6.2 ROC 곡선 그리기

In [0]:
colors = ['black', 'orange', 'blue', 'green']
linestyles = [':', '--', '-.', '-']

for clf, label, clr, ls \
        in zip(all_clf, clf_labels, colors, linestyles):

    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_true=y_test,
                                     y_score=y_pred)
    roc_auc = auc(x=fpr, y=tpr)
    plt.plot(fpr, tpr,
             color=clr,
             linestyle=ls,
             label='%s (auc = %0.3f)' % (label, roc_auc))

plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1],
         linestyle='--',
         color='gray',
         linewidth=2)

plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.grid(alpha=0.5)
plt.xlabel('False positive rate (FPR)')
plt.ylabel('True positive rate (TPR)')

plt.show()

# 아무것도 안해도 반은 맞춘다. 

### 7. 최적화
#### 7.1 하이퍼파라미터 튜닝
- 그리드 서치를 사용한 머신 러닝 모델 시부 튜닝
- 파라미터 확인


In [0]:
voting.get_params()

**Voting은 parameter 값을 찾아주는 기능이 있다.

In [0]:
params = {'logistic__C': [0.001, 0.1, 100.0],
          'tree__max_depth': [1,2,3,4,5],
          'knn__n_neighbors' : [1,3,5,7]}

grid = GridSearchCV(estimator=voting,
                    param_grid=params,
                    cv=10,
                    scoring='roc_auc',
                    iid=False)
grid.fit(X_train, y_train)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.3f %r"
    % (grid.cv_results_['mean_test_score'][r],
       grid.cv_results_['std_test_score'][r]/ 2.0,
       grid.cv_results_['params'][r]))

print('최적의 파라미터 : %s' % grid.best_params_)
print('ACU: %.3f' % grid.best_score_)